In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf

from domoku.tools import GomokuTools as Gt

In [3]:
input_size=7

In [4]:
import os
import tempfile
from pathlib import Path

tempdir = tempfile.mkdtemp()

data_dir = Path(os.environ.get('MODELS_DIR', tempdir)) / 'gomoku'
models_dir = data_dir / 'models'
logs_dir = data_dir / 'logs'
cp_dir = data_dir / 'checkpoints'
print(f"Checkpoints can be found in {cp_dir}")
print(f"Models can be found in {models_dir}")
print(f"Logs can be found in {logs_dir}")

Checkpoints can be found in /var/folders/0s/fny92xb54c50mb90g3l_zmx00000gp/T/tmpaxcqwvvl/gomoku/checkpoints
Models can be found in /var/folders/0s/fny92xb54c50mb90g3l_zmx00000gp/T/tmpaxcqwvvl/gomoku/models
Logs can be found in /var/folders/0s/fny92xb54c50mb90g3l_zmx00000gp/T/tmpaxcqwvvl/gomoku/logs


---

# Tutorial: Training a Conv Model
#### Detecting Lines of Three on a Board of 7x7

## Sampling Random Boards

In [5]:
from notebooks.ml_basics_recap.data import new_sample
sample = new_sample(board_size=input_size, num_blacks=20, num_whites=0)
sample.shape

(7, 7, 2)

In [6]:
Gt.print_bin(sample, True)

shape: (7, 7, 2)
[[0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 1. 1.]
 [1. 1. 1. 1. 0. 1. 0.]
 [1. 1. 1. 0. 0. 0. 0.]]


---

## The Detection Map

In [7]:
from notebooks.ml_basics_recap.models.heuristic_detector import HeuristicDetector
detector = HeuristicDetector(input_size)

2022-05-28 07:20:14.825970: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Notice the additional dimension for the batch size come and go

In [8]:
tf.floor(detector(np.expand_dims(sample, 0))+.01)

<tf.Tensor: shape=(1, 7, 7), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 1., 0.],
        [0., 1., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.]]], dtype=float32)>

In [9]:
np.squeeze(detector(np.expand_dims(sample, 0))).shape

(7, 7)

---

## Batches From a Dataset
#### Strong Advice: Meticulously observe the shape of all incoming data!

In [10]:
BATCH_SIZE=4

In [11]:
from notebooks.ml_basics_recap.data import new_sample, new_dataset
dataset = new_dataset(100, lambda: new_sample(input_size, 20, 0), detector).batch(BATCH_SIZE)

In [12]:
iterator  = iter(dataset)
states, labels = iterator.next()
print (f"Inputs' shape: {states.shape}, Labels' shape: {labels.shape}")

Inputs' shape: (4, 7, 7, 2), Labels' shape: (4, 7, 7)


---

## The Trainable Model

In [13]:
from notebooks.ml_basics_recap.models import SimpleConvQFunction

model_q = SimpleConvQFunction(input_size, n_filters=8, n_layers=4)
print(model_q(states).shape)
print(np.squeeze(model_q(np.expand_dims(states, 0))).shape)

(4, 7, 7, 1)
(4, 7, 7)


---

## Training

In [14]:
TRAIN_SIZE = 1024 * 8
TEST_SIZE = 1024
BATCH_SIZE = 256

### Create a dataset from the heuristics

In [15]:
train_dataset = new_dataset(size=TRAIN_SIZE,
                            sampler=lambda: new_sample(board_size=input_size, num_blacks=20, num_whites=0),
                            labeler=detector, separate=False).batch(BATCH_SIZE)

test_dataset = new_dataset(size=TEST_SIZE,
                           sampler=lambda: new_sample(board_size=input_size, num_blacks=20, num_whites=0),
                           labeler=detector, separate=False).batch(BATCH_SIZE)

In [17]:
from notebooks.ml_basics_recap.training import Trainer

trainer = Trainer(model_q, train_data=train_dataset, test_data=test_dataset,
                  logs_dir=logs_dir, models_dir=models_dir, checkpoints_dir=cp_dir)

trainer.train(20, custom_train=True)

Model: "simple_conv_q_function"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  152       
                                                                 
 conv2d_1 (Conv2D)           multiple                  584       
                                                                 
 conv2d_2 (Conv2D)           multiple                  584       
                                                                 
 conv2d_3 (Conv2D)           multiple                  584       
                                                                 
 conv2d_4 (Conv2D)           multiple                  73        
                                                                 
Total params: 1,979
Trainable params: 1,977
Non-trainable params: 2
_________________________________________________________________
Epoch 1, Loss: 0.05559470131993294, Accura

## Manual Inspection

In [18]:
BATCH_SIZE=1

In [19]:
dataset = new_dataset(10, lambda: new_sample(input_size, 20, 0), detector).batch(BATCH_SIZE)
iterator  = iter(dataset)

### Labels vs Learned

In [22]:
states, labels = iterator.next()
pred = np.squeeze(model_q(states))

print(tf.floor(labels+.01))
print(np.round(pred, 2))

tf.Tensor(
[[[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]], shape=(1, 7, 7), dtype=float32)
[[-0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0.  0. -0.]
 [-0. -0. -0. -0. -0.  1.  0.]
 [-0. -0. -0. -0. -0.  0.  0.]
 [-0. -0. -0. -0. -0.  1. -0.]
 [-0. -0. -0. -0. -0.  0. -0.]
 [-0. -0. -0. -0. -0. -0. -0.]]
